In [ ]:
from genie.conf import Genie


testbed = Genie.init('device.yaml')
device = testbed.devices['AS200']
for neighbor in device.custom['neighbors']:
    print(neighbor['asn'])


In [ ]:
import traceback

def configure_prefix_list(testbed):
        for device in testbed:
            try:
                for neighbor in device.custom['neighbors']:
                    asn = neighbor['asn']
                    as_set = get_as_set(asn)
                    assert isinstance(as_set, str)
                    assert as_set != ''

                    prefixes = get_as_set_prefixes(as_set, 4 , aggregate=True)
                    assert isinstance(prefixes, list)
                    assert len(prefixes) > 0

                    payload = create_payload(name=as_set, prefixes=prefixes)
                    device.nc.edit_config(target='running', config=payload)
                    device.changed = True
            except Exception as e:
                print(f"Failed execution on {device} and ASN {asn}")
                traceback.print_exc()

In [ ]:
configure_prefix_list(testbed)

In [ ]:
import requests
import json
def get_as_set(asn):
    request = requests.get(
            "https://www.peeringdb.com/api/net?asn=" + str(asn))
    request.raise_for_status()
    response = json.loads(request.text)
    return response['data'][0]['irr_as_set']

In [ ]:
configure_prefix_list(testbed)

In [ ]:
import subprocess

def get_as_set_prefixes(as_set, ip_version, aggregate=None,):
    args = ["bgpq4", "-j"]
    if(ip_version == 4 or ip_version == 6):
        args.append("-" + str(ip_version))
    else:
        raise Exeption("Incorrect IP version")
    if(aggregate):
        args.append("-A")
    args.append("-lirr_prefix")
    args.append(as_set)
    process = subprocess.run(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if(process.returncode != 0):
        raise Exception(f"BGPq4 failed:\n{str(process.stderr)}")
    output = json.loads(process.stdout) 
    return output['irr_prefix']

In [ ]:
configure_prefix_list(testbed)

## NETCONF Payload for prefix-list

```XML
<?xml version="1.0" ?>
<config>
	<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
		<ip>
			<prefix-list>
				<prefixes operation="replace">
					<name>AS-CISCO</name>
					<seq>
						<no>5</no>
						<action>permit</action>
						<ip>192.31.7.0/24</ip>
					</seq>
					<seq>
						<no>10</no>
						<action>permit</action>
						<ip>192.118.76.0/22</ip>
					</seq>
				</prefixes>
			</prefix-list>
		</ip>
	</native>
</config>
```

### Succesfull result

```XML
<?xml version="1.0" ?>
<rpc-reply message-id="urn:uuid:7dd3c532-92ca-4c8a-99ea-8d6b111a9bcb" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
	<ok/>
</rpc-reply>
```

### Failed result

```python
    raise RPCError()
```

In [ ]:
from pytanga.components import configComponent
from pytanga.components.Cisco.xe import nativeComponent
from pytanga.components.Cisco.xe.ip import ipComponent
from pytanga.components.Cisco.xe.ip import prefixeslistsComponent
from pytanga.helpers.Cisco.xe import ConfigurePrefixList
from pytanga.visitors import NETCONFVisitor


def create_payload(name, prefixes):
    helper = ConfigurePrefixList(name=name, replace=True)
    
    for prefix in prefixes:
        if(prefix['exact'] is True):
            helper.addPrefix(action='permit', network=prefix['prefix'])
        else:
            args = {
                'action': 'permit',
                'network': prefix['prefix']
            }
            if('less-equal' in prefix):
                args['le'] = prefix['less-equal']
            if('greater-equal' in prefix):
                args['ge'] = prefix['greater-equal']
            helper.addPrefix(**args)

    config = configComponent()
    native = nativeComponent()
    ip = ipComponent()
    prefix_list = helper.getPrefixList()
    
    config.add(native)
    native.add(ip)
    ip.add(prefix_list)
    serializer = NETCONFVisitor()
    output = config.parse(serializer)
    xml_string = serializer.print(output)
    return xml_string


In [ ]:
configure_prefix_list(testbed)

In [ ]:
for device in testbed:
        device.connect(alias='nc', via='netconf')
        
configure_prefix_list(testbed)

for device in testbed:
        device.nc.disconnect()

In [ ]:
def check_configured_object(testbed):
    for device in testbed:
        for neighbor in device.custom['neighbors']:
            asn = neighbor['asn']
            try:
                as_set = get_as_set(asn)
                assert isinstance(as_set, str)
                assert as_set != ''
                
                expected_prefixes = get_as_set_prefixes(as_set, 4, aggregate=True)
                assert isinstance(expected_prefixes, list)
                assert len(expected_prefixes) > 0
                
                configured_prefixes = get_configured_prefixes(as_set, device)
                assert isinstance(configured_prefixes, list)
                assert len(configured_prefixes) > 0
                
                assert len(expected_prefixes) == len(configured_prefixes)
                
                for prefix in configured_prefixes:
                    assert prefix in expected_prefixes
            except Exception as e:
                print(f"Failed execution on {device} and ASN {asn}")
                traceback.print_exc() 

In [ ]:
check_configured_object(testbed)

## Get configured prefix list

### Payload

```XML
<?xml version="1.0" ?>
<filter>
	<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
		<ip>
			<prefix-list>
				<prefixes>
					<name>AS-CISCO</name>
				</prefixes>
			</prefix-list>
		</ip>
	</native>
</filter>
```

### Result Object

```XML
<?xml version="1.0" ?>
<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
	<native xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-native">
		<ip>
			<prefix-list>
				<prefixes>
					<name xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">AS-CISCO</name>
					<seq>
						<no>5</no>
						<action>permit</action>
						<ip>192.31.7.0/24</ip>
					</seq>
					<seq>
						<no>10</no>
						<action>permit</action>
						<ip>192.118.76.0/22</ip>
					</seq>
				</prefixes>
			</prefix-list>
		</ip>
	</native>
</data>                   
```

In [ ]:
from pytanga.components import filterComponent
import xmltodict

def get_configured_prefixes(as_set, device):
    filter_component = filterComponent()
    native_component = nativeComponent()
    ip_component = ipComponent()
    helper = ConfigurePrefixList(name=as_set)
    prefix_list_component = helper.getPrefixList()
    
    filter_component.add(native_component)
    native_component.add(ip_component)
    ip_component.add(prefix_list_component)
    serializer = NETCONFVisitor()
    output = filter_component.parse(serializer)
    filter_payload = serializer.print(output)
    
    configured_prefixes = device.nc.get_config(
                    source='running', filter=filter_payload)
    configObject = xmltodict.parse(configured_prefixes.data_xml)
    conf_prefixes = configObject['data']['native']['ip']['prefix-list']['prefixes']['seq']
    prefixes = []
    if(isinstance(conf_prefixes, list)):
        for prefix in conf_prefixes:
            prefix_object = {
                'prefix' : prefix['ip']
            }
            if( ('le' in prefix) or ('ge' in prefix) ): 
                prefix_object['exact'] = False
                if( 'le' in prefix):
                    prefix_object['less-equal'] = int(prefix['le'])
                if( 'ge' in prefix):
                    prefix_object['greater-equal'] = int(prefix['ge'])
            else:
                prefix_object['exact'] = True
            prefixes.append(prefix_object)
    else:
        prefix = conf_prefixes
        prefix_object = {
            'prefix' : prefix['ip']
        }
        if( ('le' in prefix) or ('ge' in prefix) ): 
            prefix_object['exact'] = True
            if( 'le' in prefix):
                prefix_object['less-equal'] = int(prefix['le'])
            if( 'ge' in prefix):
                prefix_object['greater-equal'] = int(prefix['ge'])
        else:
            prefix_object['exact'] = False
        prefixes.append(prefix_object)
        
    return prefixes

In [ ]:
for device in testbed:
        device.connect(alias='nc', via='netconf')
        
check_configured_object(testbed)

for device in testbed:
        device.nc.disconnect()

In [ ]:
def check_installed_prefixes(testbed):
    for device in testbed:
        for neighbor in device.custom['neighbors']:
            try:
           
                remote_address = neighbor['remote_address']
                asn = neighbor['asn']

                as_set = get_as_set(asn)
                assert isinstance(as_set, str)
                assert as_set != ''

                expected_prefixes = get_as_set_prefixes_list(as_set, 4)
                assert isinstance(expected_prefixes, list)
                assert len(expected_prefixes) > 0

                installed_prefixes = get_neighbor_installed_prefixes('ipv4-unicast', remote_address)
                assert isinstance(installed_prefixes, list)

                assert len(expected_prefixes) >= len(installed_prefixes)

                for prefix in installed_prefixes:
                    assert prefix in expected_prefixes
                    
            except Exception as e:
                print(f"Failed execution on {device} and ASN {asn}")
                traceback.print_exc() 

In [ ]:
check_installed_prefixes(testbed)

In [ ]:
def get_as_set_prefixes_list(as_set, ip_version):
    prefixes = get_as_set_prefixes(as_set, ip_version)
    prefixes_list = []
    for prefix in prefixes:
        prefixes_list.append(prefix['prefix'])
    return prefixes_list

In [ ]:
check_installed_prefixes(testbed)

## Get neighbor received-routes

### Payload

```XML
<filter>
  <bgp-state-data>
    <bgp-route-vrfs>
      <bgp-route-vrf>
        <bgp-route-afs>
          <bgp-route-af>
           <afi-safi>ipv4-unicast</afi-safi>
            <bgp-route-neighbors>
               <bgp-route-neighbor>
                 <nbr-id>172.30.0.1</nbr-id>
               </bgp-route-neighbor>
            </bgp-route-neighbors>
          </bgp-route-af>
        </bgp-route-afs>
      </bgp-route-vrf>
    </bgp-route-vrfs>
  </bgp-state-data>
</filter>
```

### Result Object

```XML
<?xml version="1.0" ?>
<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
	<bgp-state-data xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-bgp-oper">
		<bgp-route-vrfs>
			<bgp-route-vrf>
				<bgp-route-afs>
					<bgp-route-af>
						<afi-safi xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">ipv4-unicast</afi-safi>
						<bgp-route-neighbors>
							<bgp-route-neighbor>
								<nbr-id xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">172.30.0.1</nbr-id>
								<bgp-neighbor-route-filters>
									<bgp-neighbor-route-filter>
										<nbr-fltr>bgp-nrf-post-received</nbr-fltr>
										<bgp-neighbor-route-entries>
											<bgp-neighbor-route-entry>
												<prefix>192.31.7.0/24</prefix>
												<version>63</version>
												<available-paths>1</available-paths>
												<advertised-to/>
												<bgp-neighbor-path-entries>
													<bgp-neighbor-path-entry>
														<nexthop>172.30.0.1</nexthop>
														<metric>0</metric>
														<local-pref>100</local-pref>
														<weight>0</weight>
														<as-path>109</as-path>
														<origin>origin-igp</origin>
														<path-status>
															<valid/>
															<bestpath/>
														</path-status>
														<rpki-status>rpki-not-enabled</rpki-status>
														<community/>
														<mpls-in/>
														<mpls-out/>
														<sr-profile-name/>
														<sr-binding-sid>0</sr-binding-sid>
														<sr-label-indx>0</sr-label-indx>
														<as4-path/>
														<atomic-aggregate>false</atomic-aggregate>
														<aggr-as-number>0</aggr-as-number>
														<aggr-as4-number>0</aggr-as4-number>
														<aggr-address/>
														<originator-id/>
														<cluster-list/>
														<extended-community/>
														<ext-aigp-metric>0</ext-aigp-metric>
														<path-id>0</path-id>
														<path-origin>external-path</path-origin>
													</bgp-neighbor-path-entry>
												</bgp-neighbor-path-entries>
											</bgp-neighbor-route-entry>
											<bgp-neighbor-route-entry>
												<prefix>192.118.76.0/22</prefix>
												<version>64</version>
												<available-paths>1</available-paths>
												<advertised-to/>
												<bgp-neighbor-path-entries>
													<bgp-neighbor-path-entry>
														<nexthop>172.30.0.1</nexthop>
														<metric>0</metric>
														<local-pref>100</local-pref>
														<weight>0</weight>
														<as-path>109</as-path>
														<origin>origin-igp</origin>
														<path-status>
															<valid/>
															<bestpath/>
														</path-status>
														<rpki-status>rpki-not-enabled</rpki-status>
														<community/>
														<mpls-in/>
														<mpls-out/>
														<sr-profile-name/>
														<sr-binding-sid>0</sr-binding-sid>
														<sr-label-indx>0</sr-label-indx>
														<as4-path/>
														<atomic-aggregate>false</atomic-aggregate>
														<aggr-as-number>0</aggr-as-number>
														<aggr-as4-number>0</aggr-as4-number>
														<aggr-address/>
														<originator-id/>
														<cluster-list/>
														<extended-community/>
														<ext-aigp-metric>0</ext-aigp-metric>
														<path-id>0</path-id>
														<path-origin>external-path</path-origin>
													</bgp-neighbor-path-entry>
												</bgp-neighbor-path-entries>
											</bgp-neighbor-route-entry>
										</bgp-neighbor-route-entries>
									</bgp-neighbor-route-filter>
								</bgp-neighbor-route-filters>
							</bgp-route-neighbor>
						</bgp-route-neighbors>
					</bgp-route-af>
				</bgp-route-afs>
			</bgp-route-vrf>
		</bgp-route-vrfs>
	</bgp-state-data>
</data>
```

In [ ]:
def get_neighbor_installed_prefixes(afi_safi, remote_address):
    installedPrefixes = [] 
    filter_template = """
                <filter>
                  <bgp-state-data>
                    <bgp-route-vrfs>
                      <bgp-route-vrf>
                        <bgp-route-afs>
                          <bgp-route-af>
                           <afi-safi>{afi_safi}</afi-safi>
                            <bgp-route-neighbors>
                               <bgp-route-neighbor>
                                 <nbr-id>{neighbor}</nbr-id>
                                 <bgp-neighbor-route-filters/>
                               </bgp-route-neighbor>
                            </bgp-route-neighbors>
                          </bgp-route-af>
                        </bgp-route-afs>
                      </bgp-route-vrf>
                    </bgp-route-vrfs>
                  </bgp-state-data>
                </filter>"""
    
    netconf_payload = filter_template.format(afi_safi=afi_safi, neighbor=remote_address)
    reply = device.nc.get(filter=netconf_payload)
    data = xmltodict.parse(reply.data_xml)
    
    bgpRouteFilters = data['data']['bgp-state-data']['bgp-route-vrfs']['bgp-route-vrf']['bgp-route-afs']['bgp-route-af']['bgp-route-neighbors']['bgp-route-neighbor']['bgp-neighbor-route-filters']['bgp-neighbor-route-filter']

    if('bgp-neighbor-route-entries' in bgpRouteFilters):
        prefixes = bgpRouteFilters['bgp-neighbor-route-entries']['bgp-neighbor-route-entry']
        if(isinstance(prefixes, list)):
            for prefix in prefixes:
                installedPrefixes.append(prefix['prefix'])
        else:
            installedPrefixes.append(prefixes['prefix'])

    return installedPrefixes
    

In [ ]:
for device in testbed:
        device.connect(alias='nc', via='netconf')
        
check_installed_prefixes(testbed)

for device in testbed:
        device.nc.disconnect()

In [ ]:
def create_devices_backup(testbed):
    for device in testbed:
        try:
            create_backup(device)
            backup_path = get_backup_path(device)
            device.backup_path = backup_path
        except Exception as e:
                print(f"Failed execution on {device}")
                traceback.print_exc() 

In [ ]:
create_devices_backup(testbed)

## Creating the backup

### Payload

```XML
<?xml version="1.0" ?>
<rpc message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
	<cisco-ia:checkpoint xmlns:cisco-ia="http://cisco.com/yang/cisco-ia"/>
</rpc>
```

### Succesfull result

```XML
<?xml version="1.0" ?>
<rpc-reply message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
	<result xmlns="http://cisco.com/yang/cisco-ia">Checkpoint successful</result>
</rpc-reply>
```

### Failed result

```XML
<?xml version="1.0" ?>
<rpc-reply message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
	<result xmlns="http://cisco.com/yang/cisco-ia">Checkpoint failed</result>
</rpc-reply>
```

In [ ]:
from pytanga.components import rpcComponent
from pytanga.components.Cisco.xe import saveConfigComponent
from pytanga.components.Cisco.xe import checkpointComponent
from pytanga.components.Cisco.xe import rollbackComponent

def create_backup(device):
    rpc = rpcComponent()
    ckP = checkpointComponent()
    rpc.add(ckP)
    serializer = NETCONFVisitor()
    output = rpc.parse(serializer)
    backup_payload = serializer.print(output)
    
    checkpointReply = device.nc.request(backup_payload, timeout=40)
    checkpointStatus = xmltodict.parse(checkpointReply)
    if (checkpointStatus['rpc-reply']['result']['#text'] != 'Checkpoint successful'):
        raise Exception(f"Failed to backup on {device.name}\nRPC response: {checkpointReply}")


## Get last backup

### Payload

```XML
<?xml version="1.0" ?>
<rpc message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
    <filter>
        <checkpoint-archives>
        </checkpoint-archives>
    </filter>
</rpc>
```

### Succesfull result

```XML
<?xml version="1.0" ?>
<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
	<checkpoint-archives xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-checkpoint-archive-oper">
		<max>10</max>
		<current>1</current>
		<recent>flash:configBackup-Jan--3-19-05-34.236-0</recent>
		<archives>
			<archive>
				<number>1</number>
				<name>flash:configBackup-Jan--3-19-05-34.236-0</name>
			</archive>
		</archives>
	</checkpoint-archives>
</data>
```

### Failed result

```XML
<?xml version="1.0" ?>
<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
	<checkpoint-archives xmlns="http://cisco.com/ns/yang/Cisco-IOS-XE-checkpoint-archive-oper">
		<max>10</max>
		<current>1</current>
		<recent/>
	</checkpoint-archives>
</data>
```

In [ ]:
def get_backup_path(device):
    backupFilter = '''
    <filter>
        <checkpoint-archives>
        </checkpoint-archives>
    </filter>'''
    backupReply = device.nc.get(filter=backupFilter)
    backupData = xmltodict.parse(backupReply.data_xml)
    if backupData['data']['checkpoint-archives']['recent'] is not None:
        return backupData['data']['checkpoint-archives']['recent']
    else:
        raise Exeption("Failed to retrieve last backup")

In [ ]:
for device in testbed:
        device.connect(alias='nc', via='netconf')
        
create_devices_backup(testbed)

for device in testbed:
        device.nc.disconnect()

In [ ]:
def restore_backup(testbed):
    for device in testbed:
        try:
            restore_device(device, device.backup_path)
        except Exception as e:
                print(f"Failed to restore {device}")
                traceback.print_exc() 

In [ ]:
restore_backup(testbed)

## Rollback Backup

### Payload

```XML
<?xml version="1.0" ?>
<rpc message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
	<cisco-ia:rollback xmlns:cisco-ia="http://cisco.com/yang/cisco-ia">
		<cisco-ia:target-url>flash:configuration_backup-1</cisco-ia:target-url>
	</cisco-ia:rollback>
</rpc>
```

### Succesfull result

```XML
<?xml version="1.0" ?>
<rpc-reply message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
	<result xmlns="http://cisco.com/yang/cisco-ia">Rollback successful</result>
</rpc-reply>
```

### Failed result

```XML
<?xml version="1.0" ?>
<rpc-reply message-id="" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
	<rpc-error>
		<error-type>application</error-type>
		<error-tag>invalid-value</error-tag>
		<error-severity>error</error-severity>
		<error-path xmlns:cisco-ia="http://cisco.com/yang/cisco-ia" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
            /nc:rpc/cisco-ia:rollback
        </error-path>
		<error-message xml:lang="en">
            inconsistent value: Error: Could not open file flash:configuration_backup-1 for reading
        </error-message>
		<error-info>
			<bad-element>rollback</bad-element>
		</error-info>
	</rpc-error>
</rpc-reply>
```

In [ ]:
def restore_device(device, backup_path):
    rpc = rpcComponent()
    roolback = rollbackComponent(target_url=backup_path)
    rpc.add(roolback)
    serializer = NETCONFVisitor()
    output = rpc.parse(serializer)
    rollbackPayload = serializer.print(output)
    
    reply = device.nc.request(rollbackPayload, timeout=40)
    rollbackData = xmltodict.parse(reply)
    if 'result' not in rollbackData['rpc-reply']:
        raise Exception(reply)

In [ ]:
for device in testbed:
        device.connect(alias='nc', via='netconf')
        
restore_backup(testbed)

for device in testbed:
        device.nc.disconnect()